In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_data(column_names):
    train_dataframe = pd.read_csv('./data/housing_train.txt', delim_whitespace=True, header = None)
    test_dataframe = pd.read_csv('./data/housing_test.txt', delim_whitespace=True, header = None)
    train_dataframe.columns = column_names
    test_dataframe.columns = column_names
        
    return train_dataframe, test_dataframe 

In [3]:
column_names = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']
train_data, test_data = get_data(column_names)

In [4]:
def normalize(dataset, train_len):
    
    # normalize data using shift/scale
    maxs = dataset.max()
    mins = dataset.min()
    
    for feature in dataset.columns[:-1]:        
        for i, entry in dataset.iterrows():
            dataset.at[i, feature] = (entry[feature] - mins[feature]) / (maxs[feature] - mins[feature])
            
    return dataset.iloc[:train_len], dataset.iloc[train_len:]

In [5]:
full_data = pd.concat([train_data, test_data])
train, test = normalize(full_data, len(train_data))

In [6]:
def get_thresholds(dataset, feature):
    dataset.sort_values(by=[feature])
    ts = []

    for entry in range(len(dataset) - 1):
        ts.append((dataset.iloc[entry][feature] + dataset.iloc[entry+1][feature]) / 2)

    return ts

In [7]:
def get_best_split(dataset):
    
    max_info_gain = 0
    best_feature = None
    best_threshold = None
    mse_before = get_mse(dataset)
    
    for feature in dataset.columns[:-1]:
    
        thresholds = get_thresholds(dataset, feature)
    
        for threshold in thresholds:
            left, right = split_data(dataset, feature, threshold)
            
            if len(left) == 0 or len(right) == 0:
                continue
                
            left_mse = get_mse(left)
            right_mse = get_mse(right)
            
            w = len(left) / len(dataset)
            mse_after = (w * left_mse) + ((1-w) * right_mse)
            
            info_gain = mse_before - mse_after

            if max_info_gain <= info_gain:
                max_info_gain = info_gain
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold, max_info_gain

In [8]:
class Terminal:

    def __init__(self, dataset):
        self.prediction = dataset['MEDV'].mean()

    def predict(self):
        return self.prediction

In [9]:
class Node:

    def __init__(self, feature, threshold, left_node, right_node):
        self.feature = feature
        self.threshold = threshold
        self.left_node = left_node
        self.right_node = right_node

In [10]:
def get_mse(dataset):
    
    prediction = dataset['MEDV'].mean()
    errors = []

    for i,entry in dataset.iterrows():
        errors.append(np.square(entry['MEDV'] - prediction))
    
    mse = pd.Series(errors).mean()
    return mse

In [11]:
def split_data(dataset, feature, threshold):    
    
    left = dataset[dataset[feature] < threshold]
    right = dataset[dataset[feature] >= threshold]
    
    return left, right

In [12]:
def build_tree(dataset, depth, max_depth = 4):

    best_feature, best_threshold, info_gain = get_best_split(dataset)
    
    if info_gain == 0 or depth >=4:
        return Terminal(dataset)
    
    print('Split Selected: (Feature: {}, Threshold: {}, Info Gain: {})'.format(best_feature, best_threshold, info_gain))
    
    left_data, right_data = split_data(dataset, best_feature, best_threshold)

    left_node = build_tree(left_data, depth+1)
    right_node = build_tree(right_data, depth+1)

    return Node(best_feature, best_threshold, left_node, right_node)

In [13]:
## CREATE REGRESSION TREE
model = build_tree(train, 0, max_depth = 4)

Split Selected: (Feature: RM, Threshold: 0.7344318835025868, Info Gain: 34.32118719415788)
Split Selected: (Feature: LSTAT, Threshold: 0.22723509933774835, Info Gain: 16.982560209851442)
Split Selected: (Feature: DIS, Threshold: 0.031213341941819975, Info Gain: 19.654581386992465)
Split Selected: (Feature: RM, Threshold: 0.6203295650507761, Info Gain: 7.705330887868687)
Split Selected: (Feature: NOX, Threshold: 0.5781893004115226, Info Gain: 10.210471282494826)
Split Selected: (Feature: RM, Threshold: 0.5615060356390114, Info Gain: 3.1983879586604615)
Split Selected: (Feature: LSTAT, Threshold: 0.4732339955849889, Info Gain: 8.978634903733223)
Split Selected: (Feature: PTRATIO, Threshold: 0.574468085106383, Info Gain: 32.86988647715917)
Split Selected: (Feature: RM, Threshold: 0.8091588426901706, Info Gain: 5.514242178133589)
Split Selected: (Feature: RM, Threshold: 0.7419045794213451, Info Gain: 15.992654320987654)
Split Selected: (Feature: PTRATIO, Threshold: 0.276595744680851, Info 

In [14]:
def depth(root, num_nodes):
    if isinstance(root, Terminal):
        return 0
    
    num_nodes[0] += 1
    return 1+max(depth(root.left_node, num_nodes), depth(root.right_node, num_nodes))

In [15]:
num = [0] 
x = depth(model, num)
print('Number of Nodes: {} \nDepth of Tree: {}'.format(num, x))

Number of Nodes: [14] 
Depth of Tree: 4


In [16]:
def print_tree(root, spaces):
    
    if isinstance(root, Terminal):
        print(spaces + 'prediction: {}'.format(root.predict()))
        return
    
    print(spaces + 'NODE: Feature: {}, Threshold: {}'.format(root.feature, root.threshold))
    
    print(spaces + 'Left: ')
    print_tree(root.left_node, spaces + '  ')
    print(spaces + 'Right: ')
    print_tree(root.right_node, spaces + '  ')

In [17]:
print_tree(model, '')

NODE: Feature: RM, Threshold: 0.7344318835025868
Left: 
  NODE: Feature: LSTAT, Threshold: 0.22723509933774835
  Left: 
    NODE: Feature: DIS, Threshold: 0.031213341941819975
    Left: 
      prediction: 50.0
    Right: 
      NODE: Feature: RM, Threshold: 0.6203295650507761
      Left: 
        prediction: 24.029166666666665
      Right: 
        prediction: 30.06190476190476
  Right: 
    NODE: Feature: NOX, Threshold: 0.5781893004115226
    Left: 
      NODE: Feature: RM, Threshold: 0.5615060356390114
      Left: 
        prediction: 20.174999999999997
      Right: 
        prediction: 27.52
    Right: 
      NODE: Feature: LSTAT, Threshold: 0.4732339955849889
      Left: 
        prediction: 17.22553191489362
      Right: 
        prediction: 11.224999999999998
Right: 
  NODE: Feature: PTRATIO, Threshold: 0.574468085106383
  Left: 
    NODE: Feature: RM, Threshold: 0.8091588426901706
    Left: 
      NODE: Feature: RM, Threshold: 0.7419045794213451
      Left: 
        prediction:

In [18]:
def regress(root, entry):
    
    if isinstance(root, Terminal):
        return root.predict()
    
    if entry[root.feature] < root.threshold:
        result = regress(root.left_node, entry)
    else:
        result = regress(root.right_node, entry)
        
    return result

In [19]:
def test_model(model, test_data):
    
    predictions = []
    
    for i, entry in test_data.iterrows():
        predictions.append(regress(model, entry))
    
    errors = []
    
    for i,p in enumerate(predictions):
        errors.append(np.square(test_data.iloc[i]['MEDV'] - p))
    
    mse = pd.Series(errors).mean()
    return mse, predictions

In [20]:
train_mse, preds = test_model(model, train)
print('Train MSE: {}'.format(train_mse))

Train MSE: 17.421824165973725


In [21]:
final_mse, preds = test_model(model, test)
print('Test MSE: {}'.format(final_mse))

Test MSE: 23.243058603348036
